In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import numpy as np
import os

# Step 1: Define Paths
data_dir = "C:/Users/PIYUSH LAHORI/Downloads/dataset/fruits-360_dataset_original-size/fruits-360-original-size"  # Replace with the path to the Fruit360 dataset
train_dir = os.path.join(data_dir, "Training")
val_dir = os.path.join(data_dir, "Validation")
test_dir = os.path.join(data_dir, "Test")

# Step 2: Data Generators for Training and Testing
batch_size = 32
img_height = 64
img_width = 64

# Data generator for training and validation with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # Keep validation split
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Randomly zoom in on images
    horizontal_flip=True,  # Flip images horizontally
    fill_mode="nearest",  # Fill pixels after transformations
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

val_data = train_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Data generator for testing
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,  # Use batch size 1 for real-time testing
    class_mode="categorical",
    shuffle=False,
)

# Step 3: Define and Train the Model with Regularization and Dropout
num_classes = len(train_data.class_indices)
class_indices = train_data.class_indices  # Save the mapping of class names to indices
class_labels = {v: k for k, v in class_indices.items()}  # Reverse mapping

model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            16, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),  # Add dropout to reduce overfitting

        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(
            64, (3, 3),
            activation="relu",
            kernel_regularizer=regularizers.l2(0.001),  # Add L2 regularization
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),  # Higher dropout in fully connected layers
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Add Early Stopping
early_stopping = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stopping],
)

# Save the model
model.save("fruit360_model3.h5")

Found 4993 images belonging to 24 classes.
Found 615 images belonging to 24 classes.
Found 3110 images belonging to 24 classes.


C:\Users\PIYUSH LAHORI\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


C:\Users\PIYUSH LAHORI\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


157/157 ━━━━━━━━━━━━━━━━━━━━ 90s 541ms/step - accuracy: 0.1005 - loss: 2.9690 - val_accuracy: 0.3203 - val_loss: 2.0741
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 34s 213ms/step - accuracy: 0.3204 - loss: 1.9657 - val_accuracy: 0.5317 - val_loss: 1.3638
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 33s 206ms/step - accuracy: 0.4816 - loss: 1.4204 - val_accuracy: 0.6537 - val_loss: 1.1217
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 238ms/step - accuracy: 0.6014 - loss: 1.1100 - val_accuracy: 0.6569 - val_loss: 0.9875
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 34s 210ms/step - accuracy: 0.6820 - loss: 0.8979 - val_accuracy: 0.8211 - val_loss: 0.6869
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 198ms/step - accuracy: 0.7283 - loss: 0.7718 - val_accuracy: 0.7854 - val_loss: 0.6889
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 193ms/step - accuracy: 0.7422 - loss: 0.7332 - val_accuracy: 0.8293 - val_loss: 0.5632
Epoch 8/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 199ms/step - accuracy: 0.7705 - loss: 0.6618 - val

In [5]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("fruit360_model3.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\PIYUSH~1\AppData\Local\Temp\tmp96f_1679\assets


INFO:tensorflow:Assets written to: C:\Users\PIYUSH~1\AppData\Local\Temp\tmp96f_1679\assets


Saved artifact at 'C:\Users\PIYUSH~1\AppData\Local\Temp\tmp96f_1679'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  2257396972816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257396972048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257396971856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257396972432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257396972624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257396971664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257399022224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257399023184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257399022416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2257399024528: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [11]:
# Step 4: Real-Time Camera Detection
def preprocess_image(image):
    """Preprocess the captured frame to feed into the model."""
    image = cv2.resize(image, (img_width, img_height))  # Resize to model input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype("float32") / 255.0  # Normalize to [0, 1]
    return image

# Load TFLite model for inference
interpreter = tf.lite.Interpreter(model_path="fruit360_model3.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Open the camera
cap = cv2.VideoCapture(0)  # Change to 1 if using an external camera

if not cap.isOpened():
    print("Error: Could not open the camera.")
    exit()

print("Press 'q' to quit the real-time detection.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Preprocess the frame for model input
    input_image = preprocess_image(frame)

    # Perform inference
    interpreter.set_tensor(input_details[0]["index"], input_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])
    predicted_class = np.argmax(output_data)
    confidence = np.max(output_data)

    # Display the prediction on the video feed
    predicted_label = class_labels.get(predicted_class, "Unknown")
    cv2.putText(
        frame,
        f"Fruit: {predicted_label} ({confidence*100:.2f}%)",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA,
    )

    # Show the frame
    cv2.imshow("Fruit Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit the real-time detection.


In [9]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Make predictions on a few test samples
for i in range(5):  # Test on first 5 samples
    test_image, test_label = test_data.next()
    prediction = model.predict(test_image)
    predicted_class = np.argmax(prediction)
    actual_class = np.argmax(test_label)
    print(f"Predicted: {class_labels[predicted_class]}, Actual: {class_labels[actual_class]}")


   7/3110 ━━━━━━━━━━━━━━━━━━━━ 58s 19ms/step - accuracy: 0.1561 - loss: 2.2611     

C:\Users\PIYUSH LAHORI\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3110/3110 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - accuracy: 0.9363 - loss: 0.2284
Test Accuracy: 96.82%


AttributeError: 'DirectoryIterator' object has no attribute 'next'